In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import math

In [2]:
df = pd.read_csv('train_v2.csv', nrows = 100000)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       100000 non-null  object
 1   customDimensions      100000 non-null  object
 2   date                  100000 non-null  int64 
 3   device                100000 non-null  object
 4   fullVisitorId         100000 non-null  object
 5   geoNetwork            100000 non-null  object
 6   hits                  100000 non-null  object
 7   socialEngagementType  100000 non-null  object
 8   totals                100000 non-null  object
 9   trafficSource         100000 non-null  object
 10  visitId               100000 non-null  int64 
 11  visitNumber           100000 non-null  int64 
 12  visitStartTime        100000 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 9.9+ MB


In [4]:
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [5]:
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [6]:
for cols in json_columns:
    df_norm = pd.json_normalize([json.loads(val) for val in df[cols]]).fillna(0)
    df = pd.merge(df, df_norm, left_index=True, right_index=True)

In [7]:
df = df.drop(['socialEngagementType', 'browserVersion', 'browserSize', 'operatingSystemVersion',
              'mobileDeviceBranding', 'adwordsClickInfo.criteriaParameters', 'mobileDeviceModel', 
              'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 
              'language', 'screenColors', 'screenResolution', 'metro', 'city', 'adwordsClickInfo.isVideoAd',
              'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
              'campaign', 'keyword', 'referralPath', 'adwordsClickInfo.gclId',
              'device', 'geoNetwork', 'totals', 'trafficSource', 'customDimensions', 'hits_x', 'visits'
             ], axis=1)

In [8]:
df.replace(['(not set)', 'not available in demo dataset', '(none)'], np.nan, inplace=True)
df['isTrueDirect'].replace(0, 'False', inplace=True)
df['adContent'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.adNetworkType'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.slot'].replace(0, np.nan, inplace=True)

In [9]:
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['hits_y'] = df['hits_y'].astype(int)
df['sessionQualityDim'] = df['sessionQualityDim'].astype(int)
df['timeOnSite'] = df['timeOnSite'].astype(int)
df['transactions'] = df['transactions'].astype(int)
df['transactionRevenue'] = df['transactionRevenue'].astype(int)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)
df['visitId'] = df['visitId'].astype(str)

In [10]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['weekday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   channelGrouping                 100000 non-null  object        
 1   date                            100000 non-null  datetime64[ns]
 2   fullVisitorId                   100000 non-null  object        
 3   visitId                         100000 non-null  object        
 4   visitNumber                     100000 non-null  int64         
 5   visitStartTime                  100000 non-null  int64         
 6   browser                         99999 non-null   object        
 7   operatingSystem                 99308 non-null   object        
 8   isMobile                        100000 non-null  bool          
 9   deviceCategory                  100000 non-null  object        
 10  continent                       99873 non-null   object  

In [12]:
df['fullVisitorId'].duplicated().sum()

9903

In [13]:
df_2 = df.groupby(['fullVisitorId']).agg({'channelGrouping': pd.Series.mode, 
                                         'date': 'max', 
                                         'visitId': pd.Series.mode, 
                                         'visitNumber': 'max', 
                                         'visitStartTime': 'mean',
                                         'browser': pd.Series.mode,
                                         'operatingSystem': 'max',
                                         'isMobile': pd.Series.mode,
                                         'deviceCategory': pd.Series.mode,
                                         'continent': pd.Series.mode,
                                         'subContinent': pd.Series.mode,
                                         'country': pd.Series.mode,
                                         'hits_y': 'sum',
                                         'pageviews': 'sum',
                                         'bounces': 'sum',
                                         'newVisits': 'sum',
                                         'sessionQualityDim': 'mean',
                                         'timeOnSite': 'sum',
                                         'transactions': 'sum',
                                         'transactionRevenue': 'sum',
                                         'totalTransactionRevenue': 'sum',
                                         'source': pd.Series.mode,
                                         'weekday': 'max',
                                         'day': 'max',
                                         'month': 'max',
                                         'year': 'max'
                                        }).reset_index()

In [14]:
df_2.to_csv('df_clean.csv', index=False)